### Create a GIF of Landsat images 

inspired from https://stackstac.readthedocs.io/en/latest/examples/gif.html


In [1]:
from dask.distributed import LocalCluster,Client
import dask.utils
import dask.config
import stackstac
import dask.array as da
import pystac_client
from geogif import gif, dgif
import geopandas as gpd
import planetary_computer as pc
import xarray as xr

from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry
import os

In [2]:
# port forwarding : ssh -L 8000:localhost:8787 protect
# ou
# ssh -L 9998:129.88.193.194:22 ssh-ige
# ssh -L 9999:localhost:8787 -N -p 9998 localhost
# opne localhost:9999

cluster = LocalCluster(n_workers=10,
                       threads_per_worker=2,
                       dashboard_address=8787,
                       memory_limit='6GB')
   
#cluster = LocalCluster()   
client = Client(cluster)

display(client)

2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-l4vg1zdu', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-u5pb4_ig', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-s_wsk74g', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-q0_dyqqb', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-tahxofl0', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-2s8ssng9', purging
2022-10-25 12:18:18,949 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-hl00vqx9', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 55.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38081,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 55.88 GiB
Comm: tcp://127.0.0.1:39313,Total threads: 2
Dashboard: http://127.0.0.1:33743/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:36627,


In [3]:
from dask.utils import ensure_dict, format_bytes
    
wk = client.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)

Workers= 10, Cores=20, Memory=55.88 GiB


In [4]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [6]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [15]:
# see : https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/landsat-8-c2-l2/landsat-8-c2-l2-example.ipynb

bbox =aoi.unary_union.bounds

LandsatSTAC = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

#for collection in LandsatSTAC.get_collections():
    #print(collection)

search = (
    LandsatSTAC
    .search(
        bbox=bbox,
        query =  {"eo:cloud_cover":{"lt":1}},
        datetime = "2018-07-01/2018-09-30", 
        collections = ["landsat-8-c2-l2"]
               
    )
)

In [16]:
items = pc.sign(search)
print(str(len(items))+ ' scenes found')

18 scenes found


Turn STAC items into xarray as a temporal stack, using stackstac.


In [17]:
stack = stackstac.stack(items, chunksize=(6, 19, 405, 424),bounds_latlon=bbox, epsg = 32620, resolution=50)
stack

<xarray.DataArray 'stackstac-7822722c9a8a8cef50a6c8ef8a7c6ba7' (time: 18,
                                                                band: 19,
                                                                y: 1618, x: 1696)>
dask.array<fetch_raster_window, shape=(18, 19, 1618, 1696), dtype=float64, chunksize=(6, 19, 405, 424), chunktype=numpy.ndarray>
Coordinates: (12/27)
  * time                         (time) datetime64[ns] 2018-07-02T17:48:45.25...
    id                           (time) <U31 'LC08_L2SP_037001_20180702_02_T1...
  * band                         (band) <U13 'SR_B1' 'SR_B2' ... 'SR_QA_AEROSOL'
  * x                            (x) float64 4.966e+05 4.966e+05 ... 5.814e+05
  * y                            (y) float64 9.013e+06 9.013e+06 ... 8.932e+06
    instruments                  object {'tirs', 'oli'}
    ...                           ...
    title                        (band) <U46 'Coastal/Aerosol Band (B1)' ... ...
    gsd                          (band) float64 30.0 30.0 30.0 ... 30.0 30.0
    common_name                  (band) object 'coastal' 'blue' ... None None
    center_wavelength            (band) object 0.44 0.48 0.56 ... None None None
    full_width_half_max          (band) object 0.02 0.06 0.06 ... None None None
    epsg                         int64 32620
Attributes:
    spec:        RasterSpec(epsg=32620, bounds=(496600, 8932050, 581400, 9012...
    crs:         epsg:32620
    transform:   | 50.00, 0.00, 496600.00|\n| 0.00,-50.00, 9012950.00|\n| 0.0...
    resolution:  50

In [18]:
# use common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
#stack.band

In [19]:
#See how much input data there is for just RGB. This is the amount of data we’ll end up processing
#stack.sel(band=["red", "green", "blue"])

In [20]:
# Make a bitmask---when we bitwise-and it with the data, it leaves just the 4 bits we care about
#mask_bitfields = [1, 2, 3, 4]  # dilated cloud, cirrus, cloud, cloud shadow
#bitmask = 0
#for field in mask_bitfields:
#    bitmask |= 1 << field #a = ior(a, b) is equivalent to a |= b.

#bin(bitmask)

In [21]:
#qa = stack.sel(band="QA_PIXEL").astype("uint16")
#bad = qa & bitmask  # just look at those 4 bits

#good = stack.where(bad == 0)  # mask pixels where any one of those bits are set

In [22]:
# What's the typical interval between scenes?
#good.time.diff("time").dt.days.plot.hist();

In [23]:
# keep rgb bands + Make annual median composites (`Q` means 2 quarters)
composites = stack.sel(band=["red", "green", "blue"]).resample(time="Q").median("time")
composites.ffill("time").bfill("time")
composites


<xarray.DataArray 'stackstac-7822722c9a8a8cef50a6c8ef8a7c6ba7' (time: 1,
                                                                band: 3,
                                                                y: 1618, x: 1696)>
dask.array<stack, shape=(1, 3, 1618, 1696), dtype=float64, chunksize=(1, 3, 405, 424), chunktype=numpy.ndarray>
Coordinates: (12/18)
  * band                         (band) object 'red' 'green' 'blue'
  * x                            (x) float64 4.966e+05 4.966e+05 ... 5.814e+05
  * y                            (y) float64 9.013e+06 9.013e+06 ... 8.932e+06
    instruments                  object {'oli', 'tirs'}
    view:off_nadir               int64 0
    landsat:processing_level     <U4 'L2SP'
    ...                           ...
    gsd                          (band) float64 30.0 30.0 30.0
    common_name                  (band) object 'red' 'green' 'blue'
    center_wavelength            (band) object 0.65 0.56 0.48
    full_width_half_max          (band) object 0.04 0.06 0.06
    epsg                         int64 32620
  * time                         (time) datetime64[ns] 2018-09-30
Attributes:
    spec:        RasterSpec(epsg=32620, bounds=(496600, 8932050, 581400, 9012...
    crs:         epsg:32620
    transform:   | 50.00, 0.00, 496600.00|\n| 0.00,-50.00, 9012950.00|\n| 0.0...
    resolution:  50

In [ ]:

#singleimage= composites.sel(time='2018-04-30T00:00:00.000000000').compute()
#singleimage.plot.imshow(rgb="band", robust=True)

In [24]:
ts = composites.persist()
ts_local = ts.compute()

/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/barreje/miniconda3/envs/earthgif/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [ ]:
#ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [ ]:
#ts_local.sel(time='2018-04-30T00:00:00.000000000').plot.imshow( rgb="band", robust=True)

In [13]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from xarray.plot.utils import _rescale_imshow_rgb
import numpy as np
from typing import TYPE_CHECKING, BinaryIO, cast,Literal
import ntpath
#from typing_extensions import Literal

def save_img(arr: xr.DataArray,
             timeStamp: bool = True,
             date_position: Literal["ul", "ur", "ll", "lr"] = "ul",
             date_color: tuple[int, int, int] = (255, 255, 255),
             date_bg: tuple[int, int, int]  = (0, 0, 0)):
    date_img = str(arr.time.values)[0:10]
    # Rescale
    vmin = np.nanmin(arr)
    vmax = np.nanmax(arr)
    data= _rescale_imshow_rgb(arr, vmin, vmax, True)

    # convert to U8
    u8 = (data * 255).astype("uint8").to_numpy()
    u8 = np.clip(u8, 0, 255, out=u8)
    u8 = np.moveaxis(u8, -3, -1) #??
    # Add alpha mask
    mask: np.ndarray = arr.isnull().data.any(axis=-3)
    alpha = (~mask).astype("uint8", copy=False) * 255
    frame = np.concatenate([u8, alpha[..., None]], axis=-1)

    #imgs = [Image.fromarray(frame) for frame in frames]
    img = Image.fromarray(frame)
    
     # Write timestamps onto each frame
    fnt = ImageFont.load_default()
    if timeStamp:
        
        label = date_img
        # get a drawing context
        d = ImageDraw.Draw(img)
        d = cast(ImageDraw.ImageDraw, d)

        width, height = img.size
        t_width, t_height = fnt.getsize(label)

        offset = 15
        if date_position[0] == "u":
            y = offset
        else:
            y = height - t_height - offset

        if date_position[1] == "l":
            x = offset
        else:
            x = width - t_width - offset

        if date_bg:
            d.rectangle((x, y, x + t_width, y + t_height), fill=date_bg)
        # draw text
        d.multiline_text((x, y), label, font=fnt, fill=date_color)

    
    out_filename = ntpath.basename(filename).split('.')[0]+'_'+ date_img + '.png'
    
    img.save(
        out_filename,
        format="png"
    )


In [25]:
for t in ts_local['time']:
    save_img(ts_local.sel(time=t))

/tmp/ipykernel_63084/2964229561.py:42: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  t_width, t_height = fnt.getsize(label)
